In [ ]:
import numpy as np 
import pandas as pd 
import os
import itertools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D, Conv2DTranspose, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import load_img

from tensorflow.python.keras.callbacks import TensorBoard

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import preprocess
from model_01 import get_model
import pickle
import splitdata

import keras.backend as K
import tensorflow as tf

from keras.models import load_model
import losses

In [ ]:
with open(os.path.join(splitdata.d_data, splitdata.f_train), "rb") as file:
    files = pickle.load(file)
if not files:
    raise Exception("Could not load training files!")
print("Files: ", len(files))

In [ ]:
np.random.shuffle(files)
n_valid = int(np.ceil(len(files) * 0.15))
files_train = files[n_valid:]
files_valid = files[:n_valid]
print("Files train: ", len(files_train))
print("Files valid: ", len(files_valid))

In [ ]:
model = get_model(n_kernels=32, img_height=2084, img_width=2084)
print(model.summary())

In [ ]:
model.compile(loss=losses.weighted_binary_crossentropy,
              optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.98, nesterov=True),
              metrics=['accuracy'])
#K.get_session().run(tf.global_variables_initializer())
#tensorboard = TensorBoard(log_dir="logs") 
callbacks = [
             EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1),
             ModelCheckpoint("model02.h5",
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)]
train_generator = preprocess.dataGenerator(files_train, batches=1)
valid_generator = preprocess.dataGenerator(files_valid, batches=1)
p_weights = "model02.h5"

In [ ]:
# batch_size determines the number of samples in each mini batch
# steps_per_epoch the number of batch iterations before a training epoch is considered finished.
# validation_steps
if os.path.isfile(p_weights) and True:
    model.load_weights(p_weights)
hist = model.fit_generator(generator=train_generator,
                    steps_per_epoch=10,
                    epochs=20,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator,
                    validation_steps=6,
                    max_queue_size=2)

In [ ]:
training_loss = hist.history['loss']
test_loss = hist.history['val_loss']

epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
with open(os.path.join(splitdata.d_data, splitdata.f_test), "rb") as file:
    files_test = pickle.load(file)
if not files_test:
    raise Exception("Could not load training files!")

In [ ]:
X_test, Y_test = preprocess.load_data(files_test)
X_test = X_test / 255

In [ ]:
Y_pred = model.predict(X_test, batch_size=1, verbose=1)
Y_round = np.around(Y_pred)

In [ ]:
for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_test[i], Y_test[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])